# Intelligent E-commerce Product Assistant Agent

**Authors:** [sai sandeep chenjeri, vamsi krishna velpula,vamsi alapaty]
**Competition:** [Gen AI Intensive Course Capstone 2025Q1]
**Date:** April 19, 2025

## 1. Introduction: Problem & Solution

### The Challenge of E-commerce Navigation

Modern online stores offer vast catalogs of products. For customers, navigating these extensive selections and finding exactly what they need can be overwhelming. Traditional keyword search often falls short when users have nuanced requirements or are unsure of specific product names. Companies also need efficient ways to help customers discover products and get detailed information without manual intervention.

### Our Gen AI Solution: An Intelligent Product Agent

This notebook presents an intelligent e-commerce product assistant built using Google's Gemini Gen AI models, LangChain, and LangGraph. The agent acts as a conversational interface, allowing users to interact with the product catalog using natural language.

The agent is designed to:
* Understand user queries about products.
* Search the product catalog for relevant items.
* Provide detailed information about specific products.
* Offer personalized product recommendations based on user needs.
* Manage a user's wishlist.
* Handle broader questions about product types or technologies by searching the web.
* Maintain context throughout the conversation.

By leveraging key Generative AI capabilities, this agent aims to enhance the online shopping experience, making product discovery more intuitive and engaging.

## 2. Gen AI Capabilities Demonstrated

This project showcases the following core Gen AI capabilities as required by the competition guidelines:

1.  **Agents:** We use LangGraph to orchestrate a multi-step process, allowing the LLM to decide on actions (like calling tools) based on the conversation state and manage the flow between different stages of interaction (human input, chatbot processing, tool execution).
2.  **Function Calling:** The Gemini model is equipped with a set of predefined tools (`get_catalogue`, `get_product_details`, etc.), and the LLM uses its Function Calling ability to determine which tool is needed based on the user's request and generate structured calls to invoke them.
3.  **Retrieval Augmented Generation (RAG):** For product recommendations, the agent employs a RAG pattern. It retrieves relevant product information from a vector database (ChromaDB) based on the user's query (Retrieval) and then uses the LLM to synthesize this information into a coherent recommendation response (Generation). This process also leverages **Embeddings** and **Vector Search/Store**.

We will demonstrate where each of these capabilities is implemented within the code sections below.

## 3. Setup and Data Loading

First, we install the necessary libraries and set up the environment, including retrieving the Google API key.


In [1]:
# Remove conflicting packages from the Kaggle base environment.
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
# Install langgraph and the packages used in this lab.
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7' 
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"
!pip uninstall -qqy jupyterlab kfp  # Remove unused conflicting packages
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"
import os
from kaggle_secrets import UserSecretsClient
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import Image, display
from langchain_core.messages.ai import AIMessage
from typing import Literal
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from collections.abc import Iterable
from random import randint
from langchain_core.messages.tool import ToolMessage
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry
from google.genai import types
from google import genai
from google.genai import types
from IPython.display import Markdown

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 19.7 MB/

#### Next, we load the product catalog data from Croma Electronic products dataset. We perform basic cleaning like dropping duplicates and handling missing values to ensure data quality for the agent.

In [2]:
import pandas as pd
df = pd.read_csv("/kaggle/input/croma-electronic-products-dataset/croma_products_final.csv",usecols=['name','category','features','overview','specification','price'])
df.drop_duplicates(inplace=True,subset=["name"])
df.dropna(inplace=True)
df['name'] = df['name'].str.strip()
df['name']=df['name'].str.lower()

## 4. Embeddings and Vector Store Setup
### Embedding Function and ChromaDB
#### To enable semantic search and recommendations, we create a vector database using ChromaDB. We use Google's text-embedding-004 model via a custom GeminiEmbeddingFunction to convert our product information into numerical vector representations (embeddings).

## This section demonstrates the Embeddings and Vector Search/Store capabilities.



In [4]:
client = genai.Client(api_key=GOOGLE_API_KEY)
# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]


import chromadb
DB_NAME = "ecomdb"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)


## 5. Populating the Vector Database
### We iterate through our product DataFrame and add the combined product information (name, category, features, etc.) to the ChromaDB collection. Each product is stored along with its embedding and metadata (the original DataFrame row). Batching is used to handle the data efficiently.

In [5]:
documents=[]
metadatas=[]
ids = df['name'].tolist()
batch_size = 100  # Adjust this based on experimentation

for i in range(0, len(df), batch_size):
    batch_df = df.iloc[i:i + batch_size]
    batch_documents = []
    batch_metadatas = []
    batch_ids = batch_df['name'].tolist()

    for index, row in batch_df.iterrows():
        document = row['name'] + " " + row['category'] + " " + row['features'] + " " + row['overview'] + " " + row['specification']
        batch_documents.append(document)
        batch_metadatas.append(row.to_dict())

    try:
        db.add(
            documents=batch_documents,
            metadatas=batch_metadatas,
            ids=batch_ids
        )
    except Exception as e:
        pass

## 6. Agent System Instructions (`ECOMBOT_SYSINT`)

The core behavior, persona, and constraints of the AI agent are defined through a system instruction provided to the Language Model (LLM). This instruction guides the LLM on how to respond to user queries, when to use available tools, and how to format its output.

Our `ECOMBOT_SYSINT` establishes the agent as an intelligent product recommendation and information agent for an online business. It provides explicit instructions on:

* **Persona:** Helpful, engaging, factual, and focused solely on products.
* **Allowed Topics:** Product discovery, information, and recommendations based on the available catalog.
* **Tool Usage:** Guides the LLM on *when* to use specific tools like `get_catalogue`, `get_product_details`, `check_availability`, `get_product_recommendation`, and `check_from_web`.
* **Data Handling:** Instructions to only provide information based on the available catalog and to inform the user if a product is not sold.
* **Output Formatting:** Encourages clear and structured presentation of product details.
* **Error Handling:** Guidance on how to respond if tools are unavailable (though our implementation handles this differently, the instruction informs the LLM's intent).
* **Wishlist Management:** Explicit mention of wishlist tools and their purpose.

Providing a detailed and well-structured system instruction is key to ensuring the agent behaves as intended and effectively utilizes its tools.


In [6]:
ECOMBOT_SYSINT= (
    "system",
    "You are an intelligent product recommendation and information agent for an online business selling various products."
    "A human will interact with you to discover products and learn about them. "
    "You will answer their questions about the products based on the available catalog and "
    "documentation (and only about products - no off-topic discussion, but you can chat about product features and benefits)."
    " You will also provide product recommendations based on their needs.\n\n"
    "to get the catalogue of items use get_catalogue and give the list of items available,only give the names with are exactly matching with the name in the catalogue"
    "To get detailed information about a specific product, use the get_product_details tool.Once you have the details, please present them in a clear and structured format, including the name, description, category, features, and price." 
    "To check if a product is present in catalogue, use the check_availability tool."
    "use check_from_web tool For broader questions not specific to the business's catalog (e.g., What are the benefits of OLED screens? or How to choose the right size of hiking boots?), the agent can use Google Search to provide general information and guide the user"
    "If the user wants to save a product for later use add_to_wishlist tool(only add the products which are present in the catalogue)"
    "to get items in wishlist use get_wishlist tool"
    "to give recommendation of products to user use get_product_recommendation tool, only give the names with are exactly matching with the name in the catalogue"
    "to clear items in wishlist use clear_wishlist tool "
    "When a user asks for recommendations, analyze their query and use your knowledge of the product catalog (including descriptions and features) to suggest relevant products. You can also use embeddings and similarity search if that functionality is available to you. Explain why you are recommending these products.\n\n"
    "When answering specific questions about a product, prioritize using the get_product_details tool and information retrieved from internal product documentation (if available via retrieval augmented generation). If the information is not found internally, you can consider using Google Search for general product knowledge but be cautious about the source.\n\n"
    "don't reveal information regarding tools and schema of the data of catalo"
    "Maintain a helpful and engaging tone. Do not provide personal opinions. Stick to factual information about the products.\n\n"
    "if products are bot available in catalogue, tell them we are not selling those"
    "If any of the tools are unavailable, you can break the fourth wall and tell the user that they have not implemented them yet and should keep reading to do so.",
)

WELCOME_MSG = "Welcome to the ECOMBOT . Type `q` to quit. How may I serve you today?"

## 7. Defining the Agent's Tools
### The agent needs specific capabilities to interact with the e-commerce data and external services. These capabilities are exposed to the LLM as tools using Function Calling. The LLM decides which tool to use based on the user's query.

### This section demonstrates the Function Calling capability.

## RAG Implementation Details (for recommendations)

Our product recommendation engine uses Retrieval Augmented Generation (RAG). When the user asks for recommendations (`get_product_recommendation` tool is called):

1.  The user's query is converted into a vector embedding using `text-embedding-004`.
2.  A vector search is performed in the ChromaDB to find product documents with embeddings similar to the query embedding (Retrieval).
3.  The metadata (details) of the top N most similar products are retrieved.
4.  These retrieved product details are passed back to the LLM as part of the conversation history (via `ToolMessage`).
5.  The LLM reads these product details and generates a natural language recommendation, explaining why these products are relevant based on the original query and the retrieved information (Generation).

This process allows the agent to provide recommendations that are semantically relevant to the user's request, even if they don't use exact keywords present in product names.

In [22]:
def get_catalogue() -> str:
    """gets the list of entire catalogue (names and categories)"""
    return df[['name','category']]

def get_product_details(name : str)-> str:
    """gives details of a specific product"""
    # Find the product in the DataFrame based on the name
    my_dict=df[df['name']==name.strip().lower()].to_dict(orient='records')
    return my_dict

def get_product_recommendation( query :str) -> str:
    """ Give product recommendation based on user query using similarity search."""
    # Set mode to query for embedding the user's request
    embed_fn.document_mode = False
    # Perform similarity search in ChromaDB
    results = db.query(query_texts=[query], n_results=3, include=['documents', 'metadatas'])
    # Reset mode back to document for potential future data addition (though not strictly necessary in this agent flow)
    embed_fn.document_mode = True
    return results
    

def	check_availability(name : str ) -> list:
    """ checks whether product is available in catalogue or not, returns a list of matching product names"""
    matching_products = df[df["name"].str.contains(name.strip().lower() , case=False, na=False)]['name'].tolist()
    if matching_products:
        return f"Found {len(matching_products)} matching products: {', '.join(matching_products)}"
    else:
        return f"No products found matching '{name}'."

def check_product_count(name : str) -> int:
    """ checks how many products with the name are availble"""
    return len(df[df["name"].str.contains(name , case=False, na=False, regex=False)]['name'])
    

def check_from_web(query : str) -> str:
    """Search the web for general information related to the query (e.g., benefits of OLED screens). Use for questions not answerable by the product catalog."""
    # Note: This requires a model configured with Google Search tool access.
    # Ensure your LLM initialization (`ChatGoogleGenerativeAI`) supports tools.
    # The response object needs to be processed to extract search results if necessary,
    # but we return it directly to the LLM for interpretation.
    config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())])
    try :
        response = client.models.generate_content(
            model='gemini-2.0-flash',
            contents=query,
            config=config_with_search,
        )
    except Exception as e:
         return f"Web search failed: {e}"
    return response

# Wishlist tools - these are defined using the @tool decorator,
# making them discoverable by LangChain/LangGraph's ToolNode if needed.
# In this graph, we handle them in a custom 'routing' node for state manipulation.
@tool
def add_to_wishlist(products : list[str]) -> list:
    """adds a list of product names to the wishlist. Only adds products present in the catalogue."""
    # This function will be called, but the state update happens in the routing node.
    # Its primary purpose is to define the tool for the LLM.
    pass # Implementation is in the routing node

@tool
def get_wishlist() -> list:
    """ gets the list of item names currently present in the wishlist."""
    # This function is called, but the wishlist state is managed externally.
    pass # Implementation is in the routing node

@tool
def clear_wishlist():
    """clears all items from the wishlist."""
    # This function is called, but the wishlist state is managed externally.
    pass # Implementation is in the routing node

tools = [get_catalogue,get_product_details,clear_wishlist,
         get_wishlist,check_availability,add_to_wishlist,
         get_product_recommendation,check_product_count,check_from_web]


## 8. Defining the Agent State
### In LangGraph, the agent's current situation is captured in a State. This state is passed between nodes in the graph and updated by them. Our state includes the conversation history (messages) and the user's wishlist.

In [23]:
class EcomState(TypedDict):
    """State representing the customer's order conversation."""

    # The chat conversation. This preserves the conversation history
    # between nodes. The `add_messages` annotation indicates to LangGraph
    # that state is updated by appending returned messages, not replacing
    # them.
    messages: Annotated[list, add_messages]

    # the products that customer wants to buy
    wishlist : list[str]

    # Flag indicating that the order is placed and completed.
    finished: bool


## 9. Building the Agent Graph

This is where we define the structure and flow of our agent using LangGraph. The graph consists of different nodes representing steps in the agent's process and edges defining how the agent transitions between these steps.

This section demonstrates the **Agents** capability.

### Graph Nodes

* **chatbot:** This node contains the core LLM logic. It receives messages and generates a response, potentially including tool calls.
* **human:** This node handles human interaction. It displays the bot's message and waits for the user's input. It also checks if the user wants to quit.
* **tools:** This node executes the tools defined earlier. It receives tool call messages from the chatbot and runs the corresponding functions. This uses LangGraph's built-in `ToolNode`.
* **routing:** A custom node to handle tools that directly modify the agent's state, like adding items to the wishlist.

In [24]:
def chatbot_with_tools(state: EcomState) -> EcomState:
    """The chatbot node: Processes messages and generates responses, potentially calling tools."""
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
    llm_with_tools = llm.bind_tools(tools)
    defaults = {"wishlist": [], "finished": False}

    if state["messages"]:
        new_output = llm_with_tools.invoke([ECOMBOT_SYSINT] + state["messages"])
    else:
        new_output = AIMessage(content=WELCOME_MSG)

    # Set up some defaults if not already set, then pass through the provided state,
    # overriding only the "messages" field.
    return defaults | state | {"messages": [new_output]}

def human_node(state: EcomState) -> EcomState:
    """Display the last model message to the user, and receive the user's input."""
    last_msg = state["messages"][-1]
    print("Model:", last_msg.content)

    user_input = input("User: ")

    # If it looks like the user is trying to quit, flag the conversation
    # as over.
    if user_input in {"q", "quit", "exit", "goodbye"}:
        state["finished"] = True

    return state | {"messages": [("user", user_input)]}

def route_node(state: EcomState) -> EcomState:
    """The routing node. This is where the ecom state is manipulated."""
    tool_msg = state.get("messages", [])[-1]
    wishlist = state.get("wishlist", [])
    outbound_msgs = []
    for tool_call in tool_msg.tool_calls:
        li=[]
        if tool_call["name"] == "add_to_wishlist":
            # new_list=name_corrector(tool_call["args"]["products"])
            for product in tool_call["args"]["products"]:
                if  product not in wishlist and not df[df['name']==product].empty:
                    wishlist.append(product)
                else :
                    li.append(product)
            # response = f"cant add {} to wishlist since product not present in catalogue or already present in wishlist"
            if li:
                response = f"added {wishlist} to wishlist , cant add {li} to wishlist since products are not present in catalogue or already present in wishlist"
            else:
                response = wishlist
            
        elif tool_call["name"] == "get_wishlist":
            response = wishlist

        elif tool_call["name"] == "clear_wishlist":
            wishlist.clear()
            response = None

        else:
            raise NotImplementedError(f'Unknown tool call: {tool_call["name"]}')
        # Record the tool results as tool messages.
        outbound_msgs.append(
            ToolMessage(
                content=response,
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )

    return {"messages": outbound_msgs, "wishlist": wishlist}

tool_node = ToolNode(tools)

## 10. Graph Structure and Edges

We define the flow using edges:

* The conversation starts at the `chatbot`.
* From the `chatbot`, we conditionally route:
    * To the `tools` node if the LLM calls one of the data/search tools.
    * To the `routing` node if the LLM calls one of the wishlist tools.
    * To the `human` node if the LLM provides a final text response (no tool calls).
    * To `END` if the state indicates the conversation is finished.
* Both the `tools` and `routing` nodes always return execution back to the `chatbot` so the LLM can process the tool results and generate the next user-facing response.
* From the `human` node, we conditionally route:
    * Back to the `chatbot` with the user's input.
    * To `END` if the user indicates they want to quit.

In [25]:
def maybe_exit_human_node(state: EcomState) -> Literal["chatbot", "__end__"]:
    """Route to the chatbot, unless it looks like the user is exiting."""
    if state.get("finished", False):
        return END
    else:
        return "chatbot"

def maybe_route_to_tools(state: EcomState) -> str:
    """Route between chat and tool nodes if a tool call is made."""
    if not (msgs := state.get("messages", [])):
        raise ValueError(f"No messages found when parsing state: {state}")

    msg = msgs[-1]

    if state.get("finished", False):
        # When an order is placed, exit the app. The system instruction indicates
        # that the chatbot should say thanks and goodbye at this point, so we can exit
        # cleanly.
        return END

    elif hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
        # Route to `tools` node for any automated tool calls first.
        if any(
            tool["name"] in ["check_from_web",'check_product_count','get_catalogue', 'get_product_details', 'check_availability','get_product_recommendation'] for tool in msg.tool_calls
        ):
            return "tools"
        else:
            return "routing"

    else:
        return "human"

In [26]:
graph_builder = StateGraph(EcomState)
graph_builder.add_node("chatbot", chatbot_with_tools)
graph_builder.add_node("human", human_node)
graph_builder.add_node("tools", tool_node)
graph_builder.add_node("routing", route_node)

# Chatbot -> {ordering, tools, human, END}
graph_builder.add_conditional_edges("chatbot", maybe_route_to_tools)
# Human -> {chatbot, END}
graph_builder.add_conditional_edges("human", maybe_exit_human_node)
# Tools (both kinds) always route back to chat afterwards.
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("routing", "chatbot")
# graph_builder.add_edge("tools", "routing")

graph_builder.add_edge(START, "chatbot")
graph_with_order_tools = graph_builder.compile()

# Image(graph_with_order_tools.get_graph().draw_mermaid_png())

## 11. Running the Agent

Now we can run the compiled graph to interact with the agent. The conversation will cycle between the `human` node (for user input) and the `chatbot`/`tools`/`routing` nodes (for processing and response generation).


In [31]:
from langchain_core.callbacks import StdOutCallbackHandler
# config = {"recursion_limit": 100,"callbacks": [StdOutCallbackHandler()]}
config = {"recursion_limit": 100}
# uncomment below line to run the agent

# state = graph_with_order_tools.invoke({"messages": []}, config)


# for i in state.get("messages", []):
#     print(i)
#     print("\n")

Model: Welcome to the ECOMBOT . Type `q` to quit. How may I serve you today?


User:  show me best iphones


Model: I recommend the following iPhones:

1.  **apple iphone se 2nd gen (128gb rom, 3gb ram, mxd02hn/a, black)**:  This iPhone offers the best iPhone features in a smart, efficient, and affordable package. It has a 4.7-inch Retina HD display, A13 Bionic chip, and a 12MP camera with advanced features like Portrait mode and 4K video recording.
2.  **apple iphone se (128gb rom, 3gb ram, mxd22hn/a, (product)red)**: Similar to the black version, this iPhone SE also features a 4.7-inch Retina HD display, A13 Bionic chip, and a 12MP camera. It stands out with its (Product) Red color.
3.  **apple iphone se (128gb rom, 3gb ram, mxd12hn/a, white)**: This is another version of the iPhone SE with the same features as the black and (Product) Red models, but in white color.

These iPhones are recommended because they offer a balance of performance, features, and affordability, making them excellent choices for users looking for a compact and capable iPhone.


User:  add them to wishlist


Model: OK. I've added the following products to your wishlist: apple iphone se 2nd gen (128gb rom, 3gb ram, mxd02hn/a, black), apple iphone se (128gb rom, 3gb ram, mxd22hn/a, (product)red), and apple iphone se (128gb rom, 3gb ram, mxd12hn/a, white).


...

User:  search prices of the items present in wishlist from the web


Model: I have found the approximate prices for the items in your wishlist. Please note that these prices may vary depending on the retailer, current promotions, and the condition of the items (new, used, or refurbished):

**Apple iPhones:**

*   **Apple iPhone SE (2nd Gen, 128GB, Black):** A refurbished iPhone SE 2nd gen (128GB, Black) can be found for around $120. A new one may cost you around $1,899.
*   **Apple iPhone SE (128GB, (Product) Red):** In India, the Apple iPhone SE 2020 (128GB) in Red (Model MXD22HN/A) is listed at approximately ₹44,900.
*   **Apple iPhone SE (128GB, White):** You can purchase this iPhone from Flipkart.

**Samsung TVs:**

*   **Samsung LS03T The Frame (75-inch):** In India, the offer price is ₹1,59,990.
*   **Samsung Series 9 Q95T (55-inch):** In India, the offer price is ₹1,44,900.
*   **Samsung Series 8 Q80T (65-inch):** You can purchase this TV from Croma.


User:  give me details for Apple iPhone SE (2nd Gen, 128GB, Black)


Model: Here are the details for the Apple iPhone SE (2nd Gen, 128GB, Black):

*   **Category:** Televisions & Accessories
*   **Features:** 11.93 cm (4.7"), HD, 3 GB RAM, 128 GB ROM, iOS 13, Apple A13 Bionic chip, Single Front Camera, Single Rear Camera, 1960 mAh Lithium Ion Battery, Fingerprint
*   **Overview:** Experience the best iPhone features packed in a smart, efficient, and affordable handset. The 4.7-inch HD Retina Display is backed by a durable glass and aluminum design. It is IP67 rated for dust, water, and splash resistance. The 12 MP camera offers innovative features like six studio-light effects and Smart HDR. The A13 Bionic Chip, 3 GB RAM, and 128 GB storage ensure speedy performance. It supports wireless charging and 18W fast charging.
*   **Price:** ₹44,900.00
*   **Specifications:**
    *   Mobile Category: iOS Smartphone, Touch Form Factor, New Condition
    *   Manufacturer Details: Apple, iPhone SE Series, Model Number: MXD02HN/A
    *   Operating System: iOS 13
  

User:  goodbye


## 12. Demonstrating Interactions

Here are some examples of how you can interact with the agent to showcase its capabilities. Run the cell above and use the input prompt to try these queries:

**Getting the catalogue:**
- show me the catalogue
- list all products

**Checking availability:**
- do you have iphones?
- is samsung galaxy s24 available?

**Getting product details:**
- tell me about apple iphone 14 (128gb rom, 6gb ram, mpva3hn/a blue) (Use exact name from catalogue or check availability first)
- what are the features of [a product name]?

**Getting recommendations (RAG/Embeddings):**
- recommend a good laptop for students
- I need a headphone for running
- suggest a soundbar for my TV

**Managing wishlist:**
- add apple iphone 14 (128gb rom, 6gb ram, mpva3hn/a blue) to my wishlist (Use exact name)
- add the suggested laptop and headphone to my wishlist (Requires the bot to identify the names from previous recommendations)
- show my wishlist
- clear my wishlist

**Using web search:**
- what are the benefits of noise cancelling headphones? (General knowledge not specifically in catalog)
- how to choose a microwave wattage?

To quit the conversation, type `q`, `quit`, `exit`, or `goodbye`.

## 13. Conclusion

This project successfully demonstrates the power of combining Generative AI models (Gemini) with orchestration frameworks (LangGraph) and external data sources (ChromaDB for RAG, Google Search for web lookups) to create an intelligent, conversational agent. The agent leverages Agents, Function Calling, and RAG (including Embeddings and Vector Search/Store) to provide a natural language interface for exploring an e-commerce product catalog.

This agent serves as a robust foundation for building advanced conversational e-commerce assistants.